This project will use the  Natural Language Toolkit (NLTK) to analyze the text in *Pride and Prejudice* and  *Frankenstein* find out the most common words. Common words like the, or and will be filtered out since it's obvious those sorts of words will be plentiful. Both of these novels came out in the 1810s and come from diametrically opposed genres: a novel of manners versus proto-science fiction, so it would be interesting to see how they used language differently at a surface level. 

First, the two novels will be scraped from **Project Gutenberg**. Luckily, the site offers them in plain text form already, which skips over extracting the plain text from HTML or some other format. This means that the text can be forced into all lowercase (to avoid pride and Pride being viewed as different words), and then analyzed. Analysis will include common words, sentence length, average word length, readability, and average syllables per word.

In [1]:
from bs4 import BeautifulSoup
import requests
import nltk
from collections import Counter

In [2]:
Frankenstein = requests.get("https://www.gutenberg.org/cache/epub/84/pg84.txt")
Pride = requests.get("https://www.gutenberg.org/cache/epub/1342/pg1342.txt")

FrankensteinText = Frankenstein.text.replace('\r\n', ' ').lower()[1380:]
PrideText = Pride.text.replace('\r\n', ' ').lower()[35186:]

tokenizer = nltk.tokenize.RegexpTokenizer('\w+')

FrankensteinWords = tokenizer.tokenize(FrankensteinText)
PrideWords = tokenizer.tokenize(PrideText)

FrankensteinWords[:10]

['letter',
 '1',
 '_to',
 'mrs',
 'saville',
 'england',
 '_',
 'st',
 'petersburgh',
 'dec']

The text needs to be all lowercase so "doctor" and "Doctor" would not be counted as separate words. This does include the possiblity of proper nounds being counted alongside their non-proper version (cooper the job versus Cooper the surname), but this is unlikely to skew the data too much. There are also many new line characters throughout the text than needs to be removed, alongside a lengthy section before the story. This was fairly easily done with the replace and lower methods for the first two issues. The introductory segement was removed by using a character counter and a few rounds of guessing and checking. The counter made this a simple task. 

Then a tokenizer was used to extract the words from the text, courtesy of RegEx and NLTK. This is possible without using those methods by using the split method and a space as the separator, but this is far clunkier and is more error prone.

Based on the beginning of Frankenstein there are many common words like "I", "and", "in" etc. that should be filtered out to properly compare the two texts. Luckily, NLTK has a method for this called stopwords. It is also evident some words are strangely written like "_to" and but this should have a minimal effect just like proper nouns and non-proper nouns being confused for each other. 

In [3]:
StopWords = nltk.corpus.stopwords.words('english')

StrippedFrankensteinWords = [word for word in FrankensteinWords if word not in StopWords]
StrippedPrideWords = [word for word in PrideWords if word not in StopWords]

In [4]:
StrippedFrankensteinWords[:10]

['letter',
 '1',
 '_to',
 'mrs',
 'saville',
 'england',
 '_',
 'st',
 'petersburgh',
 'dec']

In [7]:
StrippedPrideWords[:10]

['truth',
 'universally',
 'acknowledged',
 'single',
 'man',
 'possession',
 'good',
 'fortune',
 'must',
 'want']

As expected, the "_to" is kept but this shouldn't be a significant problem. Regardless, the list comprehension worked and the word lists are stripped of the stop words. Now the words can be counted and compared.

In [6]:
FrankensteinCount = Counter(StrippedFrankensteinWords)
PrideCount = Counter(StrippedPrideWords)

In [8]:
FrankensteinCount.most_common(10)

[('one', 208),
 ('could', 198),
 ('would', 184),
 ('yet', 152),
 ('man', 137),
 ('father', 133),
 ('upon', 128),
 ('life', 116),
 ('may', 113),
 ('every', 109)]

In [9]:
PrideCount.most_common(10)

[('mr', 782),
 ('elizabeth', 634),
 ('could', 524),
 ('would', 467),
 ('darcy', 418),
 ('said', 403),
 ('mrs', 343),
 ('much', 328),
 ('bennet', 327),
 ('must', 316)]

A preliminary analysis shows a general difference in tone and outlook. For example, *Pride and Prejudice* contains honorifics like Mr., Mrs. and lady amongst the most common words as well as names such as Elizabeth, Darcy, and Bennet. On the other hand, *Frankenstein* has the general terms one, father, and man as more common than specific names. This is not to see there are no clear similiarities. Both include could, would, may, much, and shall in the most used words lists. These are likely common words for the time period that happened to not be stop words in NLTK's stopwords method. There is an argument to be made that the popularity of must, much, may, might and of honorifics, titles, and names corroborates the fact that *Pride and Prejudice* is a book of manners; likewise the commonplace impersonal words like one and man corroborates the fact that *Frankenstein* is a philosophical proto-science fiction text ( this is furthered by the popularity of life,death, mind and feelings).

This is interesting to analyze, but there are other aspects of the text to analyze, such as average word and sentence length, as well as readability. The unstripped version of each work will be used for this comparison. Stop words like and would be useful for determining how long sentences and words are, as well as readability.
